In [1]:
path = 'dataset\\valid'
f_path = 'dataset\\faulty'

images = glob.glob(os.path.join(path, '*.jpg'))

rotate_methods = [cv2.cv2.ROTATE_90_CLOCKWISE, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.cv2.ROTATE_180]
rotate_name = ['c_90d.jpg', 'cc_90d.jpg', '180d.jpg']

for index, method in enumerate(rotate_methods):
	for image in images:
		n = image.split('\\')[-1].split('.')[0]
		temp = cv2.imread(image)
		temp = cv2.rotate(temp, method)
		cv2.imwrite(os.path.join(f_path, n + '_' + rotate_name[index]), temp)

In [2]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential
import numpy as np 
import cv2
import os
import matplotlib.pyplot as plt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [3]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int(f'input-unit',32,256,32), 3, input_shape=xtrain.shape[1:], activation='relu'))
    model.add(MaxPooling2D(2,2))

    for n in range(hp.Int('layer', 1, 4)):
        model.add(Conv2D(32, 3, activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer='adam',
                    loss = 'sparse_categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [5]:
categories = ['faulty','valid']
path = 'dataset'
data = []
IMG_SIZE = 100 

for category in categories:
    folder = os.path.join(path,category)
    label = categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        data.append([img_array,label])

In [6]:
len(data)

2411

In [10]:
import random 
random.shuffle(data)

In [13]:
x_data = [feature for feature, label in data]
y_data = [label for feature, label in data]

x = np.array(x_data)
x = x.astype('float32')/255
y = np.array(y_data, dtype='uint8')

In [57]:
from keras.callbacks import ReduceLROnPlateau
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 10,
    executions_per_trial = 1,
    directory='my_dir',
    project_name='Chips'
)

my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]
tuner.search(x, y, epochs = 5, batch_size=64, validation_split=0.2, callbacks= my_callbacks)

INFO:tensorflow:Reloading Oracle from existing project my_dir\Chips\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\Chips\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [58]:
tuner.results_summary()

Results summary
Results in my_dir\Chips
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
input-unit: 96
layer: 3
Score: 1.0
Trial summary
Hyperparameters:
input-unit: 32
layer: 4
Score: 1.0
Trial summary
Hyperparameters:
input-unit: 32
layer: 3
Score: 1.0
Trial summary
Hyperparameters:
input-unit: 32
layer: 1
Score: 1.0
Trial summary
Hyperparameters:
input-unit: 128
layer: 3
Score: 1.0
Trial summary
Hyperparameters:
input-unit: 160
layer: 3
Score: 0.9994813203811646
Trial summary
Hyperparameters:
input-unit: 160
layer: 2
Score: 0.9994813203811646
Trial summary
Hyperparameters:
input-unit: 64
layer: 3
Score: 0.9989626407623291
Trial summary
Hyperparameters:
input-unit: 224
layer: 2
Score: 0.9989626407623291
Trial summary
Hyperparameters:
input-unit: 192
layer: 4
Score: 0.9968879818916321


In [47]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('best_model.h5')
print('[test less, test accuracy]:', best_model.evaluate(xtest, ytest))

16/16 [==============================] - 1s 54ms/step - loss: 5.6929e-05 - accuracy: 1.0000
[test less, test accuracy]: [0.00011970717605436221, 1.0]


In [40]:
test = cv2.imread('dataset/faulty/chips (38)_cc_90d.jpg')
test = cv2.resize(test, (IMG_SIZE, IMG_SIZE))
test = test.reshape((1, IMG_SIZE, IMG_SIZE, 3))
predict = best_model.predict(test)
class_name = categories[np.argmax(predict)]
class_name

'faulty'